In [10]:
import os
import yt_dlp

def download_youtube_video_as_mp3(video_url, output_folder):
    # Set the download options for yt-dlp
    ydl_opts = {
        'format': 'bestaudio/best',  # Select the best audio quality
        'outtmpl': os.path.join(output_folder, '%(title)s.%(ext)s'),  # Output template for file name
        'postprocessors': [{  # Postprocessor to convert video to audio
            'key': 'FFmpegExtractAudio',  # Extract the audio
            'preferredcodec': 'mp3',  # Convert to mp3 format
            'preferredquality': '192',  # Quality of the mp3
        }],
    }

    # Download the video and convert it to mp3
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])
    
    print("Download and conversion to MP3 completed!")

# Example usage
video_link = "https://www.youtube.com/watch?v=hkagmGAu74Y"
output_folder = "/Users/kuljeetsinghshekhawat/Documents/coding/youtubedown/download"

download_youtube_video_as_mp3(video_link, output_folder)


[youtube] Extracting URL: https://www.youtube.com/watch?v=hkagmGAu74Y
[youtube] hkagmGAu74Y: Downloading webpage
[youtube] hkagmGAu74Y: Downloading ios player API JSON
[youtube] hkagmGAu74Y: Downloading web creator player API JSON
[youtube] hkagmGAu74Y: Downloading m3u8 information
[info] hkagmGAu74Y: Downloading 1 format(s): 251
[download] /Users/kuljeetsinghshekhawat/Documents/coding/youtubedown/download/Andrew Ng Machine Learning Career Advice.webm has already been downloaded
[download] 100% of    7.48MiB


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


DownloadError: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location

In [8]:
import os
from pytube import YouTube
import moviepy.editor as mp
import requests
import time
from api_secrets import API_KEY_ASSEMBLYAI

# AssemblyAI Endpoints and Headers
upload_endpoint = 'https://api.assemblyai.com/v2/upload'
transcript_endpoint = 'https://api.assemblyai.com/v2/transcript'

headers_auth_only = {'authorization': API_KEY_ASSEMBLYAI}

headers = {
    "authorization": API_KEY_ASSEMBLYAI,
    "content-type": "application/json"
}

CHUNK_SIZE = 5_242_880  # 5MB

# Function to download the YouTube video
def download_youtube_video(video_url, output_folder):
    try:
        yt = YouTube(video_url)
        stream = yt.streams.get_highest_resolution()
        
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        print(f"Downloading '{yt.title}' to '{output_folder}'...")
        video_file = stream.download(output_path=output_folder)
        print("Download completed!")
        return video_file
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to extract audio from video
def extract_audio(video_file, output_folder):
    try:
        video_clip = mp.VideoFileClip(video_file)
        audio_file = os.path.join(output_folder, os.path.splitext(os.path.basename(video_file))[0] + '.mp3')
        video_clip.audio.write_audiofile(audio_file)
        print(f"Audio extracted and saved to {audio_file}")
        return audio_file
    except Exception as e:
        print(f"An error occurred while extracting audio: {e}")
        return None

# Function to upload the audio file to AssemblyAI
def upload(filename):
    def read_file(filename):
        with open(filename, 'rb') as f:
            while True:
                data = f.read(CHUNK_SIZE)
                if not data:
                    break
                yield data

    upload_response = requests.post(upload_endpoint, headers=headers_auth_only, data=read_file(filename))
    return upload_response.json()['upload_url']

# Function to request transcription from AssemblyAI
def transcribe(audio_url):
    transcript_request = {
        'audio_url': audio_url
    }
    transcript_response = requests.post(transcript_endpoint, json=transcript_request, headers=headers)
    return transcript_response.json()['id']

# Polling function to check the transcription status
def poll(transcript_id):
    polling_endpoint = transcript_endpoint + '/' + transcript_id
    polling_response = requests.get(polling_endpoint, headers=headers)
    return polling_response.json()

# Function to get the transcription result
def get_transcription_result_url(url):
    transcribe_id = transcribe(url)
    while True:
        data = poll(transcribe_id)
        if data['status'] == 'completed':
            return data, None
        elif data['status'] == 'error':
            return data, data['error']
            
        print("Waiting for 30 seconds...")
        time.sleep(30)

# Function to save the transcript as a text file
def save_transcript(url, title):
    data, error = get_transcription_result_url(url)
    
    if data:
        filename = title + '.txt'
        with open(filename, 'w') as f:
            f.write(data['text'])
        print('Transcript saved as', filename)
    elif error:
        print("Error:", error)

# Example usage
video_link = "https://www.youtube.com/watch?v=4b4MUYve_U8"
output_folder = "/Users/kuljeetsinghshekhawat/Documents/coding/youtubedown/download"

# Step 1: Download the YouTube video
video_file = download_youtube_video(video_link, output_folder)

if video_file:
    # Step 2: Extract audio from the video
    audio_file = extract_audio(video_file, output_folder)
    
    if audio_file:
        # Step 3: Upload the audio file to AssemblyAI
        audio_url = upload(audio_file)
        
        # Step 4: Transcribe the audio and save the transcript
        save_transcript(audio_url, os.path.splitext(os.path.basename(audio_file))[0])


RuntimeError: No ffmpeg exe could be found. Install ffmpeg on your system, or set the IMAGEIO_FFMPEG_EXE environment variable.